# Christine's Code: CNN Introduction

In [106]:
import tensorflow as tf
import numpy as np
# from keras.preprocessing.image import ImageDataGenerator

# if you're running into issues with images at the compile step, pip install pillow.

In [107]:
# Keras now lives under TF
a = tf.keras.preprocessing.image.ImageDataGenerator()

In [108]:
# 4,000 images of dogs and 4,000 images of cats
# Total of 8,000 images

# Juypter notebook - run code via your machine with VSCode
# Google Colab will fail for highly intensive code that requires large datasets


In [109]:
# Goal is to deploy so it can differentiate between 2 different images
# Dog and Cat

In [110]:
tf.__version__

'2.17.0'

# Data Preprocessing

In [111]:
# We need to apply transformations to avoid overfitting
# high accuracy on training set, low accuracy in test set
# Why we do this - avoid overfitting

# Image Augmentation
# Your CNN does not overlearn

# Example - horizontal flips, zooms, rotations

In [112]:
# Create Preprocessor for Training Set
# Create instance, and create training class generator
training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, # feature scaling
    shear_range = 0.2, # 
    zoom_range = 0.2, # Zoom into the picture
    horizontal_flip = True # flip horizontally
    )


# Create the training set
# Flow from directory - tell it where the dataset for training is coming from
training_set = training_datagen.flow_from_directory('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 2 - CNN/training_set',
                                                 target_size = (64, 64), # affects resizing
                                                 batch_size = 32,
                                                 class_mode = 'binary') # this is a binary outcome, cat or dog

# Notice the folders are labelled dog and cat

Found 8000 images belonging to 2 classes.


In [113]:
# Create instance, and create test class generator

# DO NOT APPLY THE TRANSFORMATIONS FOR TEST-GEN EXCEPT FOR SCALING

# Test image - you do not want to apply transformations, to prevent over-learning
# i.e. not applying fit_transform() to test set, you only transform()

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 2 - CNN/test_set',
                                            target_size = (64, 64), # must be the same size and parameters here
                                            batch_size = 32,
                                            class_mode = 'binary')

# 2000 images for test set
# 8000 images for training set

Found 2000 images belonging to 2 classes.


# Part 1: Initialize CNN

In [114]:
# We need to initialize the CNN:
cnn = tf.keras.models.Sequential()

### Layer 1 - Convolution Layer = Convolution + RELU + Max Pooling

In [115]:
# Apply Convolution to have the Convolution layer:
# Three important parameters:
# Filter or feature detectors or kernels 
# Kernel Size = size of the rows and columns (square) of the size the square scanning around. The 
# Activation = always RELU function
# Input Shape = you have to specify the input shape, 3 dimensions, RGB, since it was resized, we need to do 64 by 64 by 3
# 64 by 64 is the image shape, then "3" is the RBG coloring. If it was singular color, you'd do 1.

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [116]:
# Max Pooling - second layer to consider things like spatial invariance
# Pool Size = 2 (it's square so 2 by 2)
# Strides = How many pixels the square moves around

cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides = 2))

## Add 2nd Convolution Layer

In [117]:
# Because this 2nd layer is connecting CL1 to CL2, you don't need the input shape parameter again:

# Step 1 - Create the layer itself, and specify RELU
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

# Step 2 - Max Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides = 2))

# Part 2: Flatten CNN

In [118]:
# Take CNN method under tf > keras > layers > Flatten() function
cnn.add(tf.keras.layers.Flatten())

# Part 3: Full Connection

In [119]:
# DO IT YOURSELF:

# Create the 2nd last layer before output:
cnn.add(tf.keras.layers.Dense(units = 128, activation= "relu"))

In [120]:
# Add the output layer:
cnn.add(tf.keras.layers.Dense(units= 1, activation = "sigmoid"))

# Part 4: Train the CNN

In [121]:
# Compile the CNN
cnn.compile(optimizer= "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Adam = Stochastic Gradient Descent
# Loss = Binary Cross Entropy - for binary classification problems
# Metrics = accuracy

In [122]:
# Train on Training set
# Params: x = training set, validation = test_set
# Epochs = trial and error
# Batch Size = not needed.

# pip install pillow for import error

cnn.fit(x = training_set, validation_data= test_set, epochs = 40)

Epoch 1/40


/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.5374 - loss: 0.7110 - val_accuracy: 0.6160 - val_loss: 0.6429
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.6375 - loss: 0.6376 - val_accuracy: 0.7095 - val_loss: 0.5769
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - accuracy: 0.6912 - loss: 0.5814 - val_accuracy: 0.7280 - val_loss: 0.5457
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.7171 - loss: 0.5453 - val_accuracy: 0.7570 - val_loss: 0.5047
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.7482 - loss: 0.5095 - val_accuracy: 0.7705 - val_loss: 0.5006
Epoch 6/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.7554 - loss: 0.5004 - val_accuracy: 0.7775 - val_loss: 0.4724
Epoch 7/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.7693 - loss: 0.4821 - val_accuracy: 0.7795 - val_loss: 0.4631
Epoch 8/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.7692 - loss: 0.4659 - val_accurac

# Part 5: Making Single Prediction

In [123]:
# Import numpy
from keras.preprocessing import image

### Step 1: Load the image, reduce size to 64 by 64

In [126]:
# Test image, use the specific load image function. Use Full Path.

# Make sure to pass params, such as transforming the image like image reduction - 64 by 64
image_pre = image.load_img('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 2 - CNN/test_set/dogs/corgi_1.jpg', target_size= (64,64))

### Step 2: Use PIL Format to Numpy Array for impact, to pass into CNN

In [127]:
# Convert PIL format to an array.
# It expects a 2D array to run the Neural Network
test_image =image.img_to_array(image_pre)

In [128]:
# Since the preprocessing pipeline here for "batch_size", you need to mention it in the params:
# batch size is the first, so axis = 0, or rows.
test_image = np.expand_dims(test_image, axis = 0)

In [129]:
# Result...
result = cnn.predict(test_image/255.0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [130]:
# Since we use sigmoid, the output is a probaility:
result[0][0]

0.9999982

In [131]:
# Set index, so you don't need to memorize or encode manually what the number means in result:

# This will remind you
class_key = training_set.class_indices

In [132]:
class_key

{'cats': 0, 'dogs': 1}

In [133]:
# Evaluation:
def evaluation_animal(prob):
    if prob > 0.5:
        print(f"We think this is a dog.")
    else: 
        print("We think this is a cat.")

In [134]:
evaluation_animal(result[0][0])

We think this is a dog.


# Special Notes:

* When following this code, don't forget normalization, scaling, transformation steps applied.
* Basically, if you applied XYZ to your training set, you will likely need to do something similar to test set (but don't fit).
* For special one-time predictions, you'll need to apply the transformations.
* In the course, the instructor forgot to scale by 255 in the predict() section, you need to scale it or the number will always be 1.0 (dog prediction)!

# How to prevent overfit:

1. Play around with parameters via hyperparameter tuning.
2. Use Cross-Validation sets (k-folds) so the model seems more parts of the data, generalizes better.
3. Use Regularization Techniques.
4. Increase the training set.
5. Try different things, like potentially a new model if it's not fitting well with your data, and you're often seeing better performance of training set over test set.
6. Batch size
7. Apply transformation and normalization techniques to modify your training and test data, i.e. reduce impact of outliers --> i.e. predict and divide by 255
8. Batch Normalization - another form of regularization